In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
#pip install pandas openpyxl 

In [2]:
# ruta_archivo = "C:/Users/jaemu/OneDrive - UNIR/DatosTFM/Datos completos/Datos_completos.csv"

# # Usamos pd.read_excel para leer el archivo
# df = pd.read_csv(ruta_archivo,sep=';')

# print(df.shape)

C:\Users\jaemu\AppData\Local\Temp\ipykernel_30336\733227920.py:4: DtypeWarning: Columns (21,25,26,39,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_archivo,sep=';')


(3991497, 123)


In [3]:
# #Quitar el usuario CONSUMIDOR FINAL con IdUser 2222222222222222. Porque genera similitudes erróneas entre productos adquiridos, y corresponden realmente a usuarios que decidieron no entregar su identificación.
# df_limpio = df[df['NombreUser'] != 'CONSUMIDOR FINAL']
# #Dejar sólo personas naturales en la compra
# df_limpio = df_limpio[df_limpio['CodigoNaturaleza'] == 2]
# df_limpio.shape


(3573594, 123)

In [4]:
# # Usar el mismo dataframe para calcular proporciones y muestrear
# conteo_por_fecha = df_limpio['FechaVenta'].value_counts()

# # Calcular proporciones
# total_registros = len(df_limpio)
# proporciones = conteo_por_fecha / total_registros

# # Calcular cantidad de registros por fecha para 80,000
# muestra_por_fecha = (proporciones * 50000).round().astype(int)

# # Asegurar que todas las fechas de df están en muestra_por_fecha
# muestra_por_fecha = muestra_por_fecha.reindex(df_limpio['FechaVenta'].unique(), fill_value=0)

# # Crear una muestra estratificada
# muestra = df_limpio.groupby('FechaVenta').apply(
#     lambda x: x.sample(n=int(muestra_por_fecha.loc[x.name]))
# ).reset_index(drop=True)

# # Guardar el subset en un nuevo archivo
# muestra.to_csv('C:/Users/jaemu/Documents/_DatosTFM/muestra_50k.csv', index=False)


In [5]:
# df = pd.read_csv('C:/Users/jaemu/Documents/_DatosTFM/muestra_50k.csv')
# print(df.shape)

C:\Users\jaemu\AppData\Local\Temp\ipykernel_30336\2097129638.py:1: DtypeWarning: Columns (25,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/jaemu/Documents/_DatosTFM/muestra_50k.csv')


(19896, 123)


In [6]:
# # Lista de columnas que quieres conservar
# columnas_a_conservar = [
#     'NumeroFactura', 'FechaVenta', 'Precio', 'Cantidad', 'ReferenciaProducto', 'CodigoMarca',
#     'IdUser', 'PaisUser', 'DepartamentoUser', 'MunicipioUser', 'MedioPago', 
#     'Talla', 'Color', 'Fabricante', 'ClaseProducto', 'NombreTienda', 'DepartamentoTienda', 
#     'CiudadTienda'
# ]

# # Filtrar el DataFrame conservando solo esas columnas
# df_filtrado = df[columnas_a_conservar]

# # Convertir la columna FechaVenta a tipo datetime
# df_filtrado['FechaVenta'] = pd.to_datetime(df_filtrado['FechaVenta'])
# # Extraer solo la fecha
# df_filtrado['FechaVenta'] = df_filtrado['FechaVenta'].dt.date

# print(df_filtrado.shape)
# RutaSalida = "C:/Users/jaemu/Documents/_DatosTFM/DatosLimpiosJ.csv"
# df_filtrado.to_csv(RutaSalida, index=False)


(19896, 18)


C:\Users\jaemu\AppData\Local\Temp\ipykernel_30336\3340018269.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['FechaVenta'] = pd.to_datetime(df_filtrado['FechaVenta'])
C:\Users\jaemu\AppData\Local\Temp\ipykernel_30336\3340018269.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['FechaVenta'] = df_filtrado['FechaVenta'].dt.date


In [7]:
# # Calcular el porcentaje de completitud para cada columna
# completitud = df_filtrado.notnull().mean() * 100  # Porcentaje de valores no nulos

# # Mostrar los resultados
# completitud_df = completitud.reset_index()
# completitud_df.columns = ['Columna', 'Porcentaje de Completitud (%)']
# print(completitud_df)


               Columna  Porcentaje de Completitud (%)
0        NumeroFactura                     100.000000
1           FechaVenta                     100.000000
2               Precio                     100.000000
3             Cantidad                     100.000000
4   ReferenciaProducto                     100.000000
5          CodigoMarca                     100.000000
6               IdUser                     100.000000
7             PaisUser                       8.147366
8     DepartamentoUser                       8.147366
9        MunicipioUser                       8.147366
10           MedioPago                     100.000000
11               Talla                     100.000000
12               Color                     100.000000
13          Fabricante                     100.000000
14       ClaseProducto                     100.000000
15        NombreTienda                     100.000000
16  DepartamentoTienda                     100.000000
17        CiudadTienda      

In [8]:
# #Nuevo filtro de datos según completitud
# columnas_a_conservar2 = [
#     'NumeroFactura', 'FechaVenta', 'Precio', 'Cantidad', 'ReferenciaProducto', 'CodigoMarca',
#     'IdUser', 'MedioPago', 'Talla', 'Color', 'Fabricante', 'ClaseProducto', 'NombreTienda', 'DepartamentoTienda', 
#     'CiudadTienda'
# ]

# # Filtrar el DataFrame conservando solo esas columnas
# df_filtrado2 = df_filtrado[columnas_a_conservar2]
# df_filtrado2.shape

(19896, 15)

In [9]:
# #Ahora se revisa cuántos valores únicos tiene cada columna
# # Extraer valores únicos para cada columna
# valores_unicos = {columna: df_filtrado2[columna].unique() for columna in columnas_a_conservar2}

# # Calcular el número de valores únicos y convertirlos a un DataFrame
# DatosUnicos = pd.DataFrame({
#     "nombreColumna": columnas_a_conservar2,
#     "valoresUnicos": [df_filtrado2[col].nunique() for col in columnas_a_conservar2]
# })

# # Mostrar los resultados como tabla
# print(DatosUnicos)

         nombreColumna  valoresUnicos
0        NumeroFactura          12651
1           FechaVenta            293
2               Precio            288
3             Cantidad             11
4   ReferenciaProducto           1168
5          CodigoMarca             67
6               IdUser          19570
7            MedioPago              6
8                Talla             62
9                Color           1366
10          Fabricante             51
11       ClaseProducto             27
12        NombreTienda            112
13  DepartamentoTienda             18
14        CiudadTienda             49


In [10]:
# # Anonimización de datos
# datos_anonimizados = df_filtrado2.copy()#.head(3000)
# # Anonimizar NumeroFactura 
# factUnicas = {fact_Unicas: f"fact{i+1}" for i, fact_Unicas in enumerate(datos_anonimizados['NumeroFactura'].unique())}
# datos_anonimizados['NumeroFactura'] = datos_anonimizados['NumeroFactura'].map(factUnicas)

# # Anonimizar ReferenciaProducto con etiquetas únicas (ref1, ref2, ...)
# referencias_unicas = {ref: f"ref{i+1}" for i, ref in enumerate(datos_anonimizados['ReferenciaProducto'].unique())}
# datos_anonimizados['ReferenciaProducto'] = datos_anonimizados['ReferenciaProducto'].map(referencias_unicas)

# # Anonimizar CodigoMarca con etiquetas únicas (marca1, marca2, ...)
# codigos_marca_unicos = {marca: f"marca{i+1}" for i, marca in enumerate(datos_anonimizados['CodigoMarca'].unique())}
# datos_anonimizados['CodigoMarca'] = datos_anonimizados['CodigoMarca'].map(codigos_marca_unicos)

# # Anonimizar IdUser con etiquetas únicas (id1, id2, ...)
# ids_unicos = {id_user: f"id{i+1}" for i, id_user in enumerate(datos_anonimizados['IdUser'].unique())}
# datos_anonimizados['IdUser'] = datos_anonimizados['IdUser'].map(ids_unicos)

# # Anonimizar Fabricante con etiquetas únicas (fabricante1, fabricante2, ...)
# fabricantes_unicos = {fabricante: f"fabricante{i+1}" for i, fabricante in enumerate(datos_anonimizados['Fabricante'].unique())}
# datos_anonimizados['Fabricante'] = datos_anonimizados['Fabricante'].map(fabricantes_unicos)

# # Anonimizar NombreTienda con etiquetas únicas (tienda1, tienda2, ...)
# nombres_tienda_unicos = {tienda: f"tienda{i+1}" for i, tienda in enumerate(datos_anonimizados['NombreTienda'].unique())}
# datos_anonimizados['NombreTienda'] = datos_anonimizados['NombreTienda'].map(nombres_tienda_unicos)

# # Anonimizar Color con etiquetas únicas (color1, color2, ...)
# colores_unicos = {color: f"color{i+1}" for i, color in enumerate(datos_anonimizados['Color'].unique())}
# datos_anonimizados['Color'] = datos_anonimizados['Color'].map(colores_unicos)

# # Anonimizar Talla con etiquetas únicas (talla1, talla2, ...)
# tallas_unicas = {talla: f"talla{i+1}" for i, talla in enumerate(datos_anonimizados['Talla'].unique())}
# datos_anonimizados['Talla'] = datos_anonimizados['Talla'].map(tallas_unicas)

# # Anonimizar ClaseProducto con etiquetas únicas (claseprod1, ...)
# claseprod_unicos = {talla: f"claseprod{i+1}" for i, talla in enumerate(datos_anonimizados['ClaseProducto'].unique())}
# datos_anonimizados['ClaseProducto'] = datos_anonimizados['ClaseProducto'].map(claseprod_unicos)

# # Guardar el resultado en un nuevo archivo CSV
# RutaSalida2 = "C:/Users/jaemu/Documents/_DatosTFM/DatosLimpiosAnonimizados.csv"
# datos_anonimizados.to_csv(RutaSalida2, index=False)

# datos_anonimizados.shape



(19896, 15)

In [11]:
# #Ahora se revisa cuántos valores únicos tiene cada columna para comprobar que la anonimización no haya generado diferencias en la bd
# # Extraer valores únicos para cada columna
# valores_unicos = {columna: datos_anonimizados[columna].unique() for columna in columnas_a_conservar2}

# # Calcular el número de valores únicos y convertirlos a un DataFrame
# DatosUnicos = pd.DataFrame({
#     "nombreColumna": columnas_a_conservar2,
#     "valoresUnicos": [datos_anonimizados[col].nunique() for col in columnas_a_conservar2]
# })

# # Mostrar los resultados como tabla
# print(DatosUnicos)

         nombreColumna  valoresUnicos
0        NumeroFactura          12651
1           FechaVenta            293
2               Precio            288
3             Cantidad             11
4   ReferenciaProducto           1168
5          CodigoMarca             67
6               IdUser          19570
7            MedioPago              6
8                Talla             62
9                Color           1366
10          Fabricante             51
11       ClaseProducto             27
12        NombreTienda            112
13  DepartamentoTienda             18
14        CiudadTienda             49


In [3]:
url = "https://raw.githubusercontent.com/jaemunoz8/TFM/refs/heads/main/DatosLimpiosAnonimizados.csv"
# Cargar el archivo como un DataFrame
datos_anonimizados = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(datos_anonimizados.head())

  NumeroFactura  FechaVenta    Precio  Cantidad ReferenciaProducto  \
0         fact1  2024-01-02  50411.76       1.0               ref1   
1         fact2  2024-01-02  17638.66       1.0               ref2   
2         fact3  2024-01-02      0.00       1.0               ref3   
3         fact4  2024-01-02   8394.96       1.0               ref4   
4         fact5  2024-01-02  33605.04       1.0               ref5   

  CodigoMarca IdUser          MedioPago   Talla   Color   Fabricante  \
0      marca1    id1  TARJETA DE DEBITO  talla1  color1  fabricante1   
1      marca2    id2           EFECTIVO  talla2  color2  fabricante1   
2      marca2    id3  TARJETA DE DEBITO  talla2  color3  fabricante1   
3      marca1    id4  TARJETA DE DEBITO  talla2  color4  fabricante2   
4      marca2    id5  TARJETA DE DEBITO  talla3  color5  fabricante1   

  ClaseProducto NombreTienda DepartamentoTienda  CiudadTienda  
0    claseprod1      tienda1    VALLE DEL CAUCA          CALI  
1    claseprod2   

In [4]:
#IRS BASADO EN ÍTEMS
# Seleccionar columnas relevantes para describir los productos
productos = datos_anonimizados[['ReferenciaProducto', 'CodigoMarca', 'Color', 'ClaseProducto', 'Fabricante']]

# Concatenar todas las características en un texto para cada producto creando una nueva columna en el df productos
productos['caracteristicas'] = (
    productos['CodigoMarca'].fillna('') + ' ' +
    productos['Color'].fillna('') + ' ' +
    productos['ClaseProducto'].fillna('') + ' ' +
    productos['Fabricante'].fillna('')
)

print(productos['caracteristicas'].head())
print(productos.shape)


0    marca1 color1 claseprod1 fabricante1
1    marca2 color2 claseprod2 fabricante1
2    marca2 color3 claseprod3 fabricante1
3    marca1 color4 claseprod4 fabricante2
4    marca2 color5 claseprod3 fabricante1
Name: caracteristicas, dtype: object
(19896, 6)


C:\Users\jaemu\AppData\Local\Temp\ipykernel_1604\3033102278.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  productos['caracteristicas'] = (


In [5]:
# Vectorizar las características usando TF-IDF
vectorizador = TfidfVectorizer()
matriz_vectores = vectorizador.fit_transform(productos['caracteristicas'])
print(pd.DataFrame(matriz_vectores).head(5))
matriz_vectores.shape

                                                   0
0    (0, 1444)\t0.44747980710877877\n  (0, 27)\t0...
1    (0, 1393)\t0.12394613864856978\n  (0, 1455)\...
2    (0, 1393)\t0.1707252728021752\n  (0, 1455)\t...
3    (0, 1444)\t0.20466409848160078\n  (0, 727)\t...
4    (0, 1393)\t0.21347319777838328\n  (0, 1455)\...


(19896, 1511)

In [6]:
# Calcular la similitud entre productos
similitud = cosine_similarity(matriz_vectores)
#similitud.shape

In [7]:
# Función para recomendar productos similares
def recomendar_producto(referencia_producto, similitud, productos, top_n=5):
    idx_producto = productos[productos['ReferenciaProducto'] == referencia_producto].index[0]
    indices_similares = similitud[idx_producto].argsort()[-top_n-1:-1][::-1]
    productos_recomendados = productos.iloc[indices_similares]
    return productos_recomendados, idx_producto, indices_similares

# Ejemplo de recomendación
referencia_ejemplo = 'ref20'  
recomendaciones, ind, indSimilares = recomendar_producto(referencia_ejemplo, similitud, productos)
print("Productos recomendados para el producto:", referencia_ejemplo)
recomendaciones[['ReferenciaProducto','Color','CodigoMarca','ClaseProducto','Fabricante']]

#print(ind) #Muestra el índice de la referencia seleccionada

Productos recomendados para el producto: ref20


,ReferenciaProducto,Color,CodigoMarca,ClaseProducto,Fabricante
6822,ref20,color10,marca3,claseprod6,fabricante1
2640,ref11,color10,marca3,claseprod6,fabricante1
1085,ref20,color10,marca3,claseprod6,fabricante1
1083,ref20,color10,marca3,claseprod6,fabricante1
11185,ref98,color10,marca3,claseprod6,fabricante1


In [8]:
indSimilares

array([ 6822,  2640,  1085,  1083, 11185], dtype=int64)

In [9]:
from sklearn.model_selection import train_test_split

# Crear un conjunto de prueba y entrenamiento
train, test = train_test_split(productos, test_size=0.2, random_state=42)


In [10]:
def precision_recall(referencia_producto, similitud, productos, test_set, top_n=5):
    """
    Calcula precisión y recall para un producto.
    """
    productos_recomendados, _, _ = recomendar_producto(referencia_producto, similitud, productos, top_n)
    productos_recomendados_ids = productos_recomendados['ReferenciaProducto'].values

    # Productos relevantes del test set
    relevantes = test_set[test_set['ReferenciaProducto'] == referencia_producto]
    relevantes_ids = relevantes['ReferenciaProducto'].values

    # Calcular precisión y recall
    interseccion = set(productos_recomendados_ids) & set(relevantes_ids)
    precision = len(interseccion) / top_n if top_n > 0 else 0
    recall = len(interseccion) / len(relevantes_ids) if len(relevantes_ids) > 0 else 0

    return precision, recall

def cobertura(recomendaciones, total_productos):
    """
    Calcula la cobertura de las recomendaciones.
    """
    productos_recomendados_unicos = set(recomendaciones)
    return len(productos_recomendados_unicos) / total_productos

def diversidad(recomendaciones, similitud):
    """
    Calcula la diversidad de las recomendaciones.
    """
    diversidades = []
    for recs in recomendaciones.values():
        for i in range(len(recs)):
            for j in range(i + 1, len(recs)):
                idx1, idx2 = recs[i], recs[j]
                diversidades.append(1 - similitud[idx1, idx2])  # Diversidad = 1 - similitud

    return sum(diversidades) / len(diversidades) if diversidades else 0

In [11]:
# Evaluar métricas
top_n = 5
precisiones = []
recalls = []
recomendaciones_totales = {}

for idx, producto in test.iterrows():
    referencia_producto = producto['ReferenciaProducto']
    try:
        # Generar recomendaciones
        productos_recomendados, _, indices_similares = recomendar_producto(
            referencia_producto, similitud, productos, top_n
        )
        recomendaciones_totales[referencia_producto] = indices_similares

        # Calcular precisión y recall
        precision, recall = precision_recall(referencia_producto, similitud, productos, test, top_n)
        precisiones.append(precision)
        recalls.append(recall)
    except Exception as e:
        # Si hay productos sin recomendaciones (por ejemplo, no están en el train set), ignorar
        pass

# Cobertura
total_productos = len(productos)
cobertura_global = cobertura([item for sublist in recomendaciones_totales.values() for item in sublist], total_productos)

# Diversidad
diversidad_global = diversidad(recomendaciones_totales, similitud)

# Resultados finales
print(f"Precisión@{top_n}: {sum(precisiones) / len(precisiones):.4f}")
print(f"Recall@{top_n}: {sum(recalls) / len(recalls):.4f}")
print(f"Cobertura: {cobertura_global:.4f}")
print(f"Diversidad: {diversidad_global:.4f}")


Precisión@5: 0.1265
Recall@5: 0.0754
Cobertura: 0.0694
Diversidad: 0.0449
